#### Test mynds packages in the sandbox

In [ ]:
from functools import partial
from pathlib import Path

import cv2
import onnxruntime as ort
import tqdm

from mynd.camera import CameraID
from mynd.image import Image, filter_image_clahe
from mynd.io import read_image
from mynd.visualization import KeyCode, destroy_all_windows


from mynd.visualization.stereo_visualizer import (
    StereoVisualizer,
    ImageWindow,
    ImageWindowsManager,
)


from mynd.utils.filesystem import list_directory
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


logger.info(f"Device: {ort.get_device()}")


def main() -> None:
    """Main entrypoint."""

    DIRECTORY: Path = Path(
        "/data/kingston_snv_01/acfr_images_grayworld/r23m7ms0_20140616_044549_grayworld"
    )
    PATTERN: str = "*.png"

    visualizer: StereoVisualizer = StereoVisualizer(
        stereo_methods=dict(), source=list()
    )
    window: ImageWindow = ImageWindow("my_test_window", size=(1000, 500))

    logger.info(visualizer)

    raise NotImplementedError


# INVOKE MAIN
main()